## ref : https://www.kaggle.com/aiswaryaramachandran/eda-baseline-model-0-40-rmse

In [1]:
# %%time
# import sys
# !cp ../input/rapids/rapids.0.11.0 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz
# sys.path = ["/opt/conda/envs/rapids/lib"] + ["/opt/conda/envs/rapids/lib/python3.6"] + ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

# import cudf

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sin, cos, sqrt, atan2, radians
pd.set_option('display.max_colwidth', None)

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
plt.style.use('fivethirtyeight')

import datetime as dt
from datetime import datetime
from haversine import haversine_vector

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/rapids/rapids.0.12.0
/kaggle/input/rapids/rapids.0.13.0
/kaggle/input/rapids/rapids.0.11.0
/kaggle/input/nyc-taxi-trip-duration/test.zip
/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip
/kaggle/input/nyc-taxi-trip-duration/train.zip


## Load & Preprocess data

In [3]:
train = pd.read_csv('../input/nyc-taxi-trip-duration/train.zip')
test = pd.read_csv('../input/nyc-taxi-trip-duration/test.zip')

print('train.shape :', train.shape)
print('test.shape :', test.shape)
train.head(2)

train.shape : (1458644, 11)
test.shape : (625134, 9)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663


#### 1) Convert datatype

In [4]:
train.pickup_datetime = pd.to_datetime(train.pickup_datetime, format='%Y-%m-%d %H:%M:%S')
train.dropoff_datetime = pd.to_datetime(train.dropoff_datetime, format='%Y-%m-%d %H:%M:%S')

train.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663


#### 2) Check missing values

In [5]:
print(f'Total sum of missing values : {train.isna().sum().sum()}')
train.isna().sum()

Total sum of missing values : 0


id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

#### 3) Check time period of dataset

In [6]:
print('>>> Min pickup time :', train.pickup_datetime.min())
print('>>> Max pickup time :', train.pickup_datetime.max())
print('>>> Time peroid of train dataset :', train.pickup_datetime.max() - train.pickup_datetime.min())

>>> Min pickup time : 2016-01-01 00:00:17
>>> Max pickup time : 2016-06-30 23:59:39
>>> Time peroid of train dataset : 181 days 23:59:22


#### 4) Add new feature - date, day, hour, day_of_week

In [7]:
wday_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

train['pickup_date'] = train.pickup_datetime.dt.date
train['pickup_day'] = train.pickup_datetime.dt.day
train['pickup_hour'] = train.pickup_datetime.dt.hour
train['pickup_day_of_week'] = train.pickup_datetime.dt.dayofweek.map(lambda x: wday_list[x])

train['dropoff_date'] = train.dropoff_datetime.dt.date
train['dropoff_day'] = train.dropoff_datetime.dt.day
train['dropoff_hour'] = train.dropoff_datetime.dt.hour
train['dropoff_day_of_week'] = train.dropoff_datetime.dt.dayofweek.map(lambda x: wday_list[x])

train.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,14,17,Monday,2016-03-14,14,17,Monday
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2016-06-12,12,0,Sunday,2016-06-12,12,0,Sunday


#### 5) Add new feature - Round lat, lng to 3 decimal places

In [8]:
train['pickup_latitude_round3'] = train.pickup_latitude.round(3)
train['pickup_longitude_round3'] = train.pickup_longitude.round(3)

train['dropoff_latitude_round3'] = train.dropoff_latitude.round(3)
train['dropoff_longitude_round3'] = train.dropoff_longitude.round(3)

train.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_hour,pickup_day_of_week,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week,pickup_latitude_round3,pickup_longitude_round3,dropoff_latitude_round3,dropoff_longitude_round3
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,17,Monday,2016-03-14,14,17,Monday,40.768,-73.982,40.766,-73.965
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,0,Sunday,2016-06-12,12,0,Sunday,40.739,-73.980,40.731,-73.999


#### 6) Add new feature - trip_distance(using Haversine Distance),  trip_duration_in_hour

In [9]:
train['trip_distance'] = haversine_vector(train[['pickup_latitude', 'pickup_longitude']].to_numpy(),
                                          train[['dropoff_latitude', 'dropoff_longitude']].to_numpy())
train['trip_duration_in_hour'] = train.trip_duration/3600

train.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,dropoff_date,dropoff_day,dropoff_hour,dropoff_day_of_week,pickup_latitude_round3,pickup_longitude_round3,dropoff_latitude_round3,dropoff_longitude_round3,trip_distance,trip_duration_in_hour
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,2016-03-14,14,17,Monday,40.768,-73.982,40.766,-73.965,1.498523,0.126389
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,2016-06-12,12,0,Sunday,40.739,-73.980,40.731,-73.999,1.805510,0.184167
